## Load data

In [4]:
#load data
import pandas as pd

original_data_canton_FR = pd.read_excel(r'../dataset/raw/Dep_15_Resultats_T1_complet.xlsx', sheet_name='Cantons', header=2)
# original_data_Bvot_FR        = pd.read_csv('../dataset/inputs/XDataFR_Bvot.csv', sep=';')
# original_data_Bvot_targets_FR = pd.read_csv('../dataset/labels/yDataFR_Bvot.csv', sep=';')

data_canton_FR = original_data_canton_FR.copy()
# data_Bvot_FR   = original_data_Bvot_FR.copy()
# data_Bvot_targets_FR = original_data_Bvot_targets_FR.copy()

In [5]:
################################# fonction utile #############################
def saveData(data, loc):
    # save cher_data as excel
    writer = pd.ExcelWriter(loc)
    
    # write dataframe to excel
    data.to_excel(writer)

    # save the excel
    writer.save()

############################# Format des donnees brute ##############################

def getNbBinomes(data):
    return len([header for header in data.columns if "Binôme" in header])

def explodeLines(data):
    initdf = data[['Code du département', 'Libellé du département', 'Code du canton', 
            'Libellé du canton', 'Inscrits', 'Abstentions', '% Abs/Ins', 'Votants',
            '% Vot/Ins', 'Blancs', '% Blancs/Ins', '% Blancs/Vot', 'Nuls', '% Nuls/Ins',
            '% Nuls/Vot', 'Exprimés', '% Exp/Ins', '% Exp/Vot']]

    headers = ['N°Panneau', 'Nuance', 'Binôme', 'Sièges', 'Voix', '% Voix/Ins', '% Voix/Exp']
    
    df = pd.DataFrame()
    for i in range(getNbBinomes(data)):
        partidf = data[[h+'.'+str(i) if i!=0 else h for h in headers]]
        partidf = pd.concat([initdf, partidf], axis=1)
        partidf.columns = pd.Index(initdf.columns.values.tolist() + headers)
        df = pd.concat([df, partidf])
    
    # Remove useless rows   
    df = df.dropna(how='all', subset=headers)
    
    return df

In [6]:
#explode line 
data_canton_FR = explodeLines(data_canton_FR)

## Dictionnaire des duels

In [67]:
#################################### Dictionnaire des duels #####################################

def getNuanceOfElected(data, col_siege='Sièges', col_nuance='Nuance'):
    elected = data[data[col_siege]=='Elus']
    return list(elected[col_nuance]) if len(elected)!=0 else None

def filterBestNuances(data, col_nuance='Nuance', criteria=12.50):
    bestCandidat = data[data['% Voix/Ins']>= criteria]
    
    if bestCandidat.empty or len(bestCandidat)==1:
        bestCandidat = data.sort_values(by='Voix', ascending=False).iloc[0:2,:]

    return list(bestCandidat[col_nuance])

def getDuels(data, dep, col_dep='Code du département', col_canton='Code du canton', col_siege='Sièges', col_nuance='Nuance',count=None):
    '''
        ATTENTION : data doit etre EXPLODE !
    '''
    data = data[data[col_dep]==dep]
    duels = dict()
    for canton in data[col_canton].unique():
        data_canton = data[data[col_canton]==canton]
        # allow to know if there is a majority in the canton
        elected = getNuanceOfElected(data_canton, col_siege=col_siege, col_nuance=col_nuance) 
        
        if elected is not None:
            count+=1
            duels[str(canton)]= elected
        else:
            duels[str(canton)] = filterBestNuances(data_canton)
    return (duels, count) if count is not None else duels

def optimizeDuelDict(duels):
    optdict = dict()
    for dep, duelDepDict in duels.items():
        for canton, duelList in duelDepDict.items():
            key = ':'.join(duelList)
            if key in optdict.keys():
                if dep in optdict[key].keys():
                    optdict[key][dep].append(int(canton))
                else:
                    optdict[key][dep]=[int(canton)]
            else:
                optdict[key]= dict([(dep, [int(canton)])])
    return optdict

In [68]:
#dictionnaire des duels
duels = dict()
count=0
for dep in data_canton_FR['Code du département'].unique():
    duels[str(dep)], count= getDuels(data_canton_FR, dep, count=count)
    if duels[str(dep)]==[]:
        print('empty list for dep : ', dep)

#dictionnaire ooptimize
optDuels = optimizeDuelDict(duels)
win = [duel.split(':') for duel in list(optDuels.keys()) if len(duel.split(':'))<2]


In [69]:
duels['14']
count=0
for winner in win:
    count += len(optDuels[winner[0]])
count
optDuels

': [2,
   7,
   9,
   11,
   13,
   15,
   17,
   18,
   19,
   21,
   22,
   23,
   31,
   32,
   36,
   37,
   38,
   39,
   41],
  '60': [1, 2, 3, 4, 6, 10, 11, 15, 18, 19, 20],
  '61': [1, 6, 7],
  '62': [10, 18, 24],
  '66': [14],
  '67': [4, 5, 15],
  '68': [1, 3, 4, 5, 6, 7, 8, 10, 12, 13, 14, 16],
  '69': [10, 11],
  '71': [8, 17, 29],
  '72': [7, 9, 17, 21],
  '74': [2, 16],
  '76': [8, 12, 14, 20, 21, 26, 32],
  '77': [2, 9],
  '78': [1, 7, 8],
  '80': [12, 23],
  '81': [19],
  '82': [1, 5, 10],
  '83': [2, 4, 6, 8, 9, 10, 11, 12, 13, 14, 15, 17, 19, 20, 21, 22, 23],
  '84': [15],
  '85': [8, 9, 14, 16, 17],
  '88': [5],
  '89': [2, 6, 7, 8, 9, 12, 13, 18],
  '91': [8, 17],
  '94': [16, 18, 22],
  '95': [6, 10, 11, 17]},
 'BC-UG:BC-UD:BC-FN': {'1': [2, 4, 7, 13, 16, 18, 23],
  '2': [4, 6, 11, 12, 16, 17],
  '7': [11, 15],
  '13': [5],
  '14': [24],
  '16': [4, 17],
  '17': [2],
  '21': [4, 7, 10, 16, 18, 20, 23],
  '26': [11],
  '27': [4, 17],
  '28': [7],
  '30': [19, 20],
 

## Preparation des donnees pour le reseau de neuronne

In [81]:
#################################### Data Processing ####################################

def prepareInputDataExploded(data):
    tmp = data[['NUMTOUR', 'CODDPT', 'CODSUBCOM', 'LIBSUBCOM', 'CODBURVOT', 'CODCAN',
            'LIBCAN', 'NBRINS', 'NBRVOT', 'NBREXP', 'CODNUA', 'NBRVOIX']].copy()

    # correction = [str(i) for i in range(1,10)]

    # #remove canton where there is a winner in the 1st turn
    # for winner in win:
    #      for dep, can in optDuels[winner[0]]:
    #         if dep in correction:
    #             tmp = tmp.loc[~((tmp['CODDPT']=='0'+dep) & (tmp['CODCAN']==int(can)))]
    #         else:
    #             tmp = tmp.loc[~((tmp['CODDPT']==dep) & (tmp['CODCAN']==int(can)))]
    


    # Compute missing data
    tmp['NBRABS'] = tmp['NBRINS'] - tmp['NBRVOT']
    tmp['NBRBLCNUL'] = tmp['NBRVOT'] - tmp['NBREXP']
    tmp['%ABS/INS'] = tmp['NBRABS'] / tmp['NBRINS']
    tmp['%BLCNUL/VOT'] = tmp['NBRBLCNUL'] / tmp['NBRVOT']
    tmp['%EXP/VOT'] = tmp['NBREXP'] / tmp['NBRVOT']
    tmp['%VOIX/EXP'] = tmp['NBRVOIX'] / tmp['NBREXP']

    nuances = getAllNuances(data)
    statsFeatures = ['NBRINS', 'NBREXP', '%ABS/INS', '%BLCNUL/VOT', '%EXP/VOT']
    idFeatures = ['CODDPT', 'CODCAN', 'CODSUBCOM', 'CODBURVOT']

    exprimes = tmp[idFeatures + ['NBREXP']].drop_duplicates().sort_values(idFeatures)['NBREXP']
    stats = tmp[idFeatures + statsFeatures].drop_duplicates()[statsFeatures]
    ids = tmp[idFeatures].drop_duplicates()

    # Create [%Voix] and fill it
    voix = pd.DataFrame(0, index=data.index, columns=nuances)
    for parti in nuances:
        voix[parti][data['CODNUA']==parti] = tmp[tmp['CODNUA']==parti]['NBRVOIX']
    voix = pd.concat([tmp[idFeatures], voix], axis=1).groupby(idFeatures).sum()[nuances]
    voix.index = exprimes.index

    # Concat with computed stats and divide almost everything by Exprimés
    voix = voix.divide(exprimes, axis=0)
    X = pd.concat([stats, voix], axis=1)
    X.index = pd.MultiIndex.from_frame(ids)
    return X.sort_values(['CODDPT', 'CODCAN', 'CODSUBCOM', 'CODBURVOT'])

def getAllNuances(data, colNuance='CODNUA', fmt='exploded'):
    if fmt not in ['exploded', 'line']:
        raise ValueError("format parameter must be 'exploded' or 'line'")
    
    if fmt == 'exploded':
        nuances = data[colNuance].unique()
    
    if fmt == 'line':
        nuances = np.array([])
        nuances_tmp = data[colNuance].fillna(0)
        for c in nuances_tmp:
            nuances = np.append(nuances, nuances_tmp[c])
        nuances = np.unique(nuances[nuances!=0])
    
    return sorted(nuances)

# retourne un dataset associe a un duel
def extractDuelRaw(X, y, duel, col_canton='CODCAN', col_dep='CODDPT'):
    correction = [str(i) for i in range(1,10)]
    X_duel = pd.DataFrame()
    y_duel = pd.DataFrame()

    #remove canton where there is a winner in the 1st turn
    for dep, cantonList in optDuels[':'.join(duel)].items():
        X_duel = pd.concat([X_duel, X.loc[( X[col_dep] == (dep if dep not in correction else '0'+dep) ) & ( X[col_canton].isin(cantonList) )]])
        y_duel = pd.concat([y_duel, y.loc[( y[col_dep] == (dep if dep not in correction else '0'+dep) ) & ( y[col_canton].isin(cantonList) )]])
    
    return (X_duel, y_duel)

def prepareLabelsExploded(data, oneHotEncode=False):
    nuances = getAllNuances(data)
    idFeatures = ['CODDPT', 'CODCAN', 'CODSUBCOM', 'CODBURVOT']

    exprimes = data[idFeatures+['NBREXP']].groupby(idFeatures).first()

    # Create [%Voix] and fill it
    voix = pd.DataFrame(0, index=data.index, columns=nuances)
    for parti in nuances:
        voix[parti][data['CODNUA']==parti] = data[data['CODNUA']==parti]['NBRVOIX']
    voix = pd.concat([data[idFeatures], voix], axis=1).groupby(idFeatures).sum().sort_values(idFeatures)[nuances]

    # Concat with computed stats and divide voix by exprimes
    y = voix.divide(exprimes['NBREXP'], axis=0)
    return y

In [76]:
optDuels['BC-SOC:BC-UD'].items()

dict_items([('4', [6]), ('5', [5]), ('7', [10]), ('12', [16]), ('14', [5, 6, 8, 16]), ('15', [2]), ('17', [18]), ('18', [3, 4]), ('19', [19]), ('22', [15, 18, 22, 26]), ('23', [5, 12, 14]), ('24', [5, 6, 8, 16, 18, 23]), ('25', [8]), ('26', [2, 18]), ('29', [1, 2, 4, 6, 8, 9, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26, 27]), ('31', [7, 15, 18, 19, 20, 21, 22, 23, 25, 26]), ('32', [3]), ('33', [3, 5, 6, 8, 23, 28, 29, 31]), ('34', [16]), ('35', [3, 4, 13, 18, 19]), ('37', [7, 10, 15, 19]), ('40', [1, 2, 3, 5, 6, 8, 9, 10, 11, 13, 14]), ('43', [4, 8, 13, 17]), ('44', [1, 4, 11, 12, 13, 14, 15, 16, 21, 22, 24, 25, 26, 29, 31]), ('49', [2, 3, 5, 6, 7, 17]), ('50', [7, 8]), ('54', [12, 19, 22]), ('55', [2]), ('56', [14]), ('57', [26]), ('59', [27]), ('61', [2]), ('63', [11]), ('64', [1, 5, 6, 15, 18]), ('65', [10]), ('66', [13]), ('67', [17, 20, 21]), ('71', [2, 7]), ('72', [10]), ('76', [22, 28, 29, 30]), ('77', [20, 22]), ('78', [10]), ('81', [1]), ('82', [8]), ('85', [5, 13])

In [77]:
dtypes = {
    'NUMTOUR' :    'int64',
    'CODDPT' :    'object',
    'CODSUBCOM' :  'int64',
    'LIBSUBCOM' : 'object',
    'CODBURVOT' : 'object',
    'CODCAN' :     'int64',
    'LIBCAN' :    'object',
    'NBRINS' :     'int64',
    'NBRVOT' :     'int64',
    'NBREXP' :     'int64',
    'NUMDEPCAND' : 'int64',
    'LIBLISEXT' : 'object',
    'CODNUA' :    'object',
    'NBRVOIX' :    'int64',
}

#load data
dataBvot = pd.read_csv('../dataset/raw/DP15_Bvot_T1T2.csv', delimiter=';', dtype=dtypes)
dataT1Bvot = dataBvot[dataBvot.NUMTOUR==1]
dataT2Bvot = dataBvot[dataBvot.NUMTOUR==2]

dataT1Bvot
dataT2Bvot.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 131891 entries, 289173 to 421063
Data columns (total 14 columns):
NUMTOUR       131891 non-null int64
CODDPT        131891 non-null object
CODSUBCOM     131891 non-null int64
LIBSUBCOM     131891 non-null object
CODBURVOT     131891 non-null object
CODCAN        131891 non-null int64
LIBCAN        131891 non-null object
NBRINS        131891 non-null int64
NBRVOT        131891 non-null int64
NBREXP        131891 non-null int64
NUMDEPCAND    131891 non-null int64
LIBLISEXT     131891 non-null object
CODNUA        131891 non-null object
NBRVOIX       131891 non-null int64
dtypes: int64(8), object(6)
memory usage: 15.1+ MB


In [83]:
#Selecting data

X_duel, y_duel  = extractDuelRaw(dataT1Bvot, dataT2Bvot , ['BC-SOC', 'BC-UD'] )


In [86]:
#prepare les donnees
from sklearn.model_selection import train_test_split

print('Preparing input data... ', end='')
X = prepareInputDataExploded(X_duel)

print('Preparing labels... ', end='')
y = prepareLabelsExploded(y_duel)

print(f'shape X = {X.shape} shape y = {y.shape}')

# selected_feature_X = ['NBRINS', 'NBREXP',
#        '%ABS/INS', '%BLCNUL/VOT', '%EXP/VOT', 'BC-DLF', 'BC-DVD', 'BC-DVG',
#        'BC-FG', 'BC-FN', 'BC-SOC', 'BC-UC', 'BC-UD', 'BC-UDI', 'BC-UG',
#        'BC-UMP']

selected_feature_y = ['BC-SOC', 'BC-UD'] 

y = y[selected_feature_y]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,train_size=0.8, random_state=42)
print(f'shape xtrain {X_train.shape} shape ytrain {y_train.shape} || shape xtest {X_test.shape} shape ytest {y_test.shape}')

Preparing input data... Preparing labels... shape X = (4492, 22) shape y = (4492, 2)
shape xtrain (3593, 22) shape ytrain (3593, 2) || shape xtest (899, 22) shape ytest (899, 2)


## build model

In [ ]:
tf.keras.backend.clear_session()

model = tf.keras.Sequential()
model.add(tf.keras.layers.InputLayer(input_shape=(160,)))
model.add(tf.keras.layers.Dense(64, activation='relu', kernel_initializer='he_uniform'))
model.add(tf.keras.layers.Dense(20, activation='softmax'))

opt = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [25]:
history = model.fit(trainset, y, batch_size=32, validation_data=(testset, y), epochs=100, verbose=0)

# evaluate the model
_, train_acc = model.evaluate(trainset, y, verbose=0)
_, test_acc = model.evaluate(testset, y, verbose=0)
print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))
# plot loss during training
pyplot.subplot(211)
pyplot.title('Loss')
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
# plot accuracy during training
pyplot.subplot(212)
pyplot.title('Accuracy')
pyplot.plot(history.history['accuracy'], label='train')
pyplot.plot(history.history['val_accuracy'], label='test')
pyplot.legend()
pyplot.show()

NameError: name 'model' is not defined